In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score, recall_score

import warnings
warnings.filterwarnings('ignore')

In [7]:
diabetes_dataset = pd.read_csv('../data/processed/diabetes_processed.csv')

In [9]:
diabetes_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    float64
 2   BloodPressure             768 non-null    float64
 3   SkinThickness             768 non-null    float64
 4   Insulin                   768 non-null    float64
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(6), int64(3)
memory usage: 54.1 KB


In [10]:
diabetes_dataset['Outcome'] = diabetes_dataset['Outcome'].astype('category')

In [18]:
#Seperating the features and target variable
X = diabetes_dataset.drop('Outcome', axis=1)
y = diabetes_dataset['Outcome']

In [19]:
# Scaling the features
scaler = StandardScaler()
X_norm = scaler.fit_transform(X)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.3, random_state=42)

In [23]:
LR = LogisticRegression().fit(X_train,y_train)

In [ ]:
# Make predictions
y_pred = LR.predict(X_test)
y_proba = LR.predict_proba(X_test)[:, 1]  # Probability estimates for ROC-AUC

In [25]:
# Evaluate the model
roc_auc = roc_auc_score(y_test, y_proba)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"ROC-AUC: {roc_auc:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1 Score: {f1:.3f}")

ROC-AUC: 0.798
Recall: 0.588
F1 Score: 0.610
